# Importing libraries and data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('googleplaystore.csv')
df.sample(5)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
7758,Webmail web mobile app,TOOLS,2.3,395,2.6M,"100,000+",Free,0,Everyone,Tools,"August 5, 2018",4.2,4.1 and up
4947,AdClear Ad blocker for Samsung,TOOLS,4.2,1586,1.5M,"500,000+",Free,0,Everyone,Tools,"September 7, 2017",1.0.5,5.0 and up
641,The DJ - Match with People,DATING,NaN,0,9.8M,50+,Free,0,Mature 17+,Dating,"August 2, 2018",1.2,4.1 and up
3192,Skyscanner,TRAVEL_AND_LOCAL,4.5,481546,29M,"10,000,000+",Free,0,Everyone,Travel & Local,"August 6, 2018",5.48,4.4 and up
7504,FREE CHATS C.L.,COMMUNICATION,NaN,4,3.4M,10+,Paid,$0.99,Mature 17+,Communication,"February 12, 2018",11,4.0.3 and up


In [3]:
df['Size'].str[-1].unique()

array(['M', 'e', 'k', '+'], dtype=object)

# Data Cleaning

## 1.1 Investigation

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


In [5]:
df.duplicated().sum()

np.int64(483)

In [6]:
df.describe()

,Rating
count,9367.000000
mean,4.193338
std,0.537431
min,1.000000
25%,4.000000
50%,4.300000
75%,4.500000
max,19.000000


#### Notes
- 483 duplicated row
- `Price`, `Size`, and `Android Ver` columns are mixed with string 
    - `Size` column is in multiple units
    - `Size` and `Android Ver` columns have rows with the content "Varies with device"
- `Last Updated` column needs to be converted to date
- `Reviews` column needs to be converted to integer
- `Installs` column needs to be converted to integer with removing '+'
- `Current Ver` column won't be converted to float since it is refers to the development of each app indinvidually

## 1.2 Solving data issues

### Dropping duplicated Rows

In [7]:
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df.duplicated().sum()

np.int64(0)

### Working with the Size column


In [8]:
df['Size'].value_counts()

Size
Varies with device    1526
11M                    188
13M                    186
12M                    186
14M                    182
                      ... 
552k                     1
885k                     1
1020k                    1
582k                     1
619k                     1
Name: count, Length: 462, dtype: int64

In [9]:
df[df['Size'].str.endswith('e')].sample(5)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
2941,Evie Launcher,PERSONALIZATION,4.7,139258,Varies with device,"5,000,000+",Free,0,Everyone,Personalization,"August 3, 2018",Varies with device,4.4 and up
2304,Snapchat,SOCIAL,4.0,17014705,Varies with device,"500,000,000+",Free,0,Teen,Social,"July 30, 2018",Varies with device,Varies with device
8663,Madden NFL Football,FAMILY,4.5,1455952,Varies with device,"10,000,000+",Free,0,Everyone,Sports;Action & Adventure,"May 15, 2018",4.3.6,4.0 and up
250,join.me - Simple Meetings,BUSINESS,4.0,6989,Varies with device,"1,000,000+",Free,0,Everyone,Business,"July 16, 2018",4.3.0.508,4.4 and up
8619,Drugs Dz - Algeria,MEDICAL,4.4,187,Varies with device,"10,000+",Free,0,Everyone,Medical,"July 29, 2017",0.0.7,Varies with device


In [10]:
df[df['Size'].str.endswith('+')]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
9990,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN


#### So i have noticed that this row is shifted to the right starting from the category 

In [11]:
# Fixing the shift in the row by keeping the first value which is the name then add null value to the category then the rest of the row
row = df.iloc[9990].copy()
new_row = [row[0]] + [np.nan] + list(row[1:-1])   # Takes from the second value to the one before last since the last is null
df.iloc[9990] = new_row

C:\Users\Youssif\AppData\Local\Temp\ipykernel_12060\903078688.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  new_row = [row[0]] + [np.nan] + list(row[1:-1])   # Takes from the second value to the one before last since the last is null
C:\Users\Youssif\AppData\Local\Temp\ipykernel_12060\903078688.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.9' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.iloc[9990] = new_row


In [12]:
df.iloc[9990]

App               Life Made WI-Fi Touchscreen Photo Frame
Category                                              NaN
Rating                                                1.9
Reviews                                              19.0
Size                                                 3.0M
Installs                                           1,000+
Type                                                 Free
Price                                                   0
Content Rating                                   Everyone
Genres                                                NaN
Last Updated                            February 11, 2018
Current Ver                                        1.0.19
Android Ver                                    4.0 and up
Name: 9990, dtype: object

- now all the values ends with 'e' which is varies with device will be replaced with NaN
- the size column will be converted to float
- The whole column will be in 'M' unit
    - M = K / 1024

In [13]:
def cleanSize(size):
    if size.endswith('e'):
        size = np.nan
        return size
    if size.endswith('k'):
        size = float(size.strip('k')) / 1024
        return size
    size = float(size.strip('M'))
    return size

df['Size'] = df['Size'].apply(cleanSize)
df['Size'].sample(20)

3142     11.0
7727     20.0
438      56.0
3491     63.0
8980     48.0
6300      3.8
8360      2.9
5800      3.8
6641     15.0
1498     69.0
1950     92.0
3632     13.0
1898     22.0
10086    17.0
8557     40.0
5780     43.0
1824     13.0
1752     99.0
1013     68.0
9516      5.8
Name: Size, dtype: float64

In [14]:
df["Size"].isna().sum()

np.int64(1526)

- Now all the values of the Size column are float and in 'M' unit
- There is 1527 null values in the Size column

### Converting the rest of numerical columns to float

In [15]:
df['Price'].value_counts()
# this column is mostly zeros (Exactly 88%) but i will keep it cause it may give insights on other columns like intalls or rating


Price
0          9593
$0.99       146
$2.99       125
$1.99        73
$4.99        70
           ... 
$3.61         1
$394.99       1
$1.26         1
$1.20         1
$1.04         1
Name: count, Length: 92, dtype: int64

In [16]:
df["Price"].unique()

array(['0', '$4.99', '$3.99', '$6.99', '$1.49', '$2.99', '$7.99', '$5.99',
       '$3.49', '$1.99', '$9.99', '$7.49', '$0.99', '$9.00', '$5.49',
       '$10.00', '$24.99', '$11.99', '$79.99', '$16.99', '$14.99',
       '$1.00', '$29.99', '$12.99', '$2.49', '$10.99', '$1.50', '$19.99',
       '$15.99', '$33.99', '$74.99', '$39.99', '$3.95', '$4.49', '$1.70',
       '$8.99', '$2.00', '$3.88', '$25.99', '$399.99', '$17.99',
       '$400.00', '$3.02', '$1.76', '$4.84', '$4.77', '$1.61', '$2.50',
       '$1.59', '$6.49', '$1.29', '$5.00', '$13.99', '$299.99', '$379.99',
       '$37.99', '$18.99', '$389.99', '$19.90', '$8.49', '$1.75',
       '$14.00', '$4.85', '$46.99', '$109.99', '$154.99', '$3.08',
       '$2.59', '$4.80', '$1.96', '$19.40', '$3.90', '$4.59', '$15.46',
       '$3.04', '$4.29', '$2.60', '$3.28', '$4.60', '$28.99', '$2.95',
       '$2.90', '$1.97', '$200.00', '$89.99', '$2.56', '$30.99', '$3.61',
       '$394.99', '$1.26', '$1.20', '$1.04'], dtype=object)

> the `Price` column is mostly zeros (Exactly 88%) but i will keep it cause it may give insights on other columns like installs or rating

In [17]:
# Converting Price to float
df["Price"] = df["Price"].apply(lambda x : float(x.strip('$')))
df["Price"].unique()

array([  0.  ,   4.99,   3.99,   6.99,   1.49,   2.99,   7.99,   5.99,
         3.49,   1.99,   9.99,   7.49,   0.99,   9.  ,   5.49,  10.  ,
        24.99,  11.99,  79.99,  16.99,  14.99,   1.  ,  29.99,  12.99,
         2.49,  10.99,   1.5 ,  19.99,  15.99,  33.99,  74.99,  39.99,
         3.95,   4.49,   1.7 ,   8.99,   2.  ,   3.88,  25.99, 399.99,
        17.99, 400.  ,   3.02,   1.76,   4.84,   4.77,   1.61,   2.5 ,
         1.59,   6.49,   1.29,   5.  ,  13.99, 299.99, 379.99,  37.99,
        18.99, 389.99,  19.9 ,   8.49,   1.75,  14.  ,   4.85,  46.99,
       109.99, 154.99,   3.08,   2.59,   4.8 ,   1.96,  19.4 ,   3.9 ,
         4.59,  15.46,   3.04,   4.29,   2.6 ,   3.28,   4.6 ,  28.99,
         2.95,   2.9 ,   1.97, 200.  ,  89.99,   2.56,  30.99,   3.61,
       394.99,   1.26,   1.2 ,   1.04])

In [18]:
# Converting Reviews column to float
df["Reviews"] = df['Reviews']. apply(lambda x : int(x))
df["Reviews"]

0           159
1           967
2         87510
3        215644
4           967
          ...  
10353        38
10354         4
10355         3
10356       114
10357    398307
Name: Reviews, Length: 10358, dtype: int64

In [19]:
df["Android Ver"].unique()

array(['4.0.3 and up', '4.2 and up', '4.4 and up', '2.3 and up',
       '3.0 and up', '4.1 and up', '4.0 and up', '2.3.3 and up',
       'Varies with device', '2.2 and up', '5.0 and up', '6.0 and up',
       '1.6 and up', '1.5 and up', '2.1 and up', '7.0 and up',
       '5.1 and up', '4.3 and up', '4.0.3 - 7.1.1', '2.0 and up',
       '3.2 and up', '4.4W and up', '7.1 and up', '7.0 - 7.1.1',
       '8.0 and up', '5.0 - 8.0', '3.1 and up', '2.0.1 and up',
       '4.1 - 7.1.1', nan, '5.0 - 6.0', '1.0 and up', '2.2 - 7.1.1',
       '5.0 - 7.1.1'], dtype=object)

> Here i will create a new column which contains only the minimum android version needed

> Varies with device will be replaced with NaN

In [20]:
df["Android Ver"] = df["Android Ver"].replace("Varies with device", np.nan)
df["min android ver"] = df["Android Ver"].str.split().str[0]

In [21]:
df["min android ver"].unique()

array(['4.0.3', '4.2', '4.4', '2.3', '3.0', '4.1', '4.0', '2.3.3', nan,
       '2.2', '5.0', '6.0', '1.6', '1.5', '2.1', '7.0', '5.1', '4.3',
       '2.0', '3.2', '4.4W', '7.1', '8.0', '3.1', '2.0.1', '1.0'],
      dtype=object)

In [22]:
df.loc[df["min android ver"] == '4.4W'].count()

App                11
Category           11
Rating              5
Reviews            11
Size               11
Installs           11
Type               11
Price              11
Content Rating     11
Genres             11
Last Updated       11
Current Ver        11
Android Ver        11
min android ver    11
dtype: int64

> The android version "4.4W" is a valid android version which is 4.4 but for smart watches so i will convert it to normal 4.4 since they are only 11 rows out of 10841 row 

In [23]:
# fixing the value 4.4W
df["min android ver"] = df["min android ver"].apply(lambda x: x.strip('W') if isinstance(x, str) else x)
df["min android ver"].unique()

array(['4.0.3', '4.2', '4.4', '2.3', '3.0', '4.1', '4.0', '2.3.3', nan,
       '2.2', '5.0', '6.0', '1.6', '1.5', '2.1', '7.0', '5.1', '4.3',
       '2.0', '3.2', '7.1', '8.0', '3.1', '2.0.1', '1.0'], dtype=object)

> Converting it to float with neglecting the third number in the version since it is only the patch level not the version

In [24]:
df["min android ver"] = df["min android ver"].apply(lambda x: float('.'.join(x.split('.')[:2])) if isinstance(x, str) else x)
df["min android ver"].unique()

array([4. , 4.2, 4.4, 2.3, 3. , 4.1, nan, 2.2, 5. , 6. , 1.6, 1.5, 2.1,
       7. , 5.1, 4.3, 2. , 3.2, 7.1, 8. , 3.1, 1. ])

> Converting rating to float

In [25]:
df["Rating"] = df["Rating"].astype(float)
df["Rating"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 10358 entries, 0 to 10357
Series name: Rating
Non-Null Count  Dtype  
--------------  -----  
8893 non-null   float64
dtypes: float64(1)
memory usage: 81.1 KB


> Converting Installs column in another column called `installs range`

In [26]:
df["Installs"].unique()

array(['10,000+', '500,000+', '5,000,000+', '50,000,000+', '100,000+',
       '50,000+', '1,000,000+', '10,000,000+', '5,000+', '100,000,000+',
       '1,000,000,000+', '1,000+', '500,000,000+', '50+', '100+', '500+',
       '10+', '1+', '5+', '0+', '0'], dtype=object)

In [27]:
df["installs range"] = df["Installs"].str.strip('+').str.replace(',', '').astype(int)
df["installs range"]

0           10000
1          500000
2         5000000
3        50000000
4          100000
           ...   
10353        5000
10354         100
10355        1000
10356        1000
10357    10000000
Name: installs range, Length: 10358, dtype: int64

### Converting `Last Updated` to date

In [28]:
df["Last Updated"] = pd.to_datetime(df["Last Updated"])
df["Last Updated"]

0       2018-01-07
1       2018-01-15
2       2018-08-01
3       2018-06-08
4       2018-06-20
           ...    
10353   2017-07-25
10354   2018-07-06
10355   2017-01-20
10356   2015-01-19
10357   2018-07-25
Name: Last Updated, Length: 10358, dtype: datetime64[ns]

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10358 entries, 0 to 10357
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   App              10358 non-null  object        
 1   Category         10357 non-null  object        
 2   Rating           8893 non-null   float64       
 3   Reviews          10358 non-null  int64         
 4   Size             8832 non-null   float64       
 5   Installs         10358 non-null  object        
 6   Type             10357 non-null  object        
 7   Price            10358 non-null  float64       
 8   Content Rating   10358 non-null  object        
 9   Genres           10357 non-null  object        
 10  Last Updated     10358 non-null  datetime64[ns]
 11  Current Ver      10350 non-null  object        
 12  Android Ver      9135 non-null   object        
 13  min android ver  9135 non-null   float64       
 14  installs range   10358 non-null  int64

In [30]:
df.describe()

,Rating,Reviews,Size,Price,Last Updated,min android ver,installs range
count,8893.000000,1.035800e+04,8832.000000,10358.000000,10358,9135.000000,1.035800e+04
mean,4.187619,4.058654e+05,21.285343,1.030701,2017-11-14 09:37:38.428268032,3.853432,1.415639e+07
min,1.000000,0.000000e+00,0.008301,0.000000,2010-05-21 00:00:00,1.000000,0.000000e+00
25%,4.000000,3.200000e+01,4.700000,0.000000,2017-09-03 00:00:00,4.000000,1.000000e+03
50%,4.300000,1.678500e+03,13.000000,0.000000,2018-05-20 00:00:00,4.100000,1.000000e+05
75%,4.500000,4.640425e+04,29.000000,0.000000,2018-07-19 00:00:00,4.100000,1.000000e+06
max,5.000000,7.815831e+07,100.000000,400.000000,2018-08-08 00:00:00,8.000000,1.000000e+09
std,0.522910,2.696651e+06,22.540155,16.277843,NaN,0.842739,8.023580e+07


### Checking Consistency in categorical columns

##### Checking `Category` with it's nulls

In [31]:
df["Category"].unique()

array(['ART_AND_DESIGN', 'AUTO_AND_VEHICLES', 'BEAUTY',
       'BOOKS_AND_REFERENCE', 'BUSINESS', 'COMICS', 'COMMUNICATION',
       'DATING', 'EDUCATION', 'ENTERTAINMENT', 'EVENTS', 'FINANCE',
       'FOOD_AND_DRINK', 'HEALTH_AND_FITNESS', 'HOUSE_AND_HOME',
       'LIBRARIES_AND_DEMO', 'LIFESTYLE', 'GAME', 'FAMILY', 'MEDICAL',
       'SOCIAL', 'SHOPPING', 'PHOTOGRAPHY', 'SPORTS', 'TRAVEL_AND_LOCAL',
       'TOOLS', 'PERSONALIZATION', 'PRODUCTIVITY', 'PARENTING', 'WEATHER',
       'VIDEO_PLAYERS', 'NEWS_AND_MAGAZINES', 'MAPS_AND_NAVIGATION', nan],
      dtype=object)

In [32]:
df.loc[df["Category"].isna()]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,min android ver,installs range
9990,Life Made WI-Fi Touchscreen Photo Frame,NaN,1.9,19,3.0,"1,000+",Free,0.0,Everyone,NaN,2018-02-11,1.0.19,4.0 and up,4.0,1000


##### Checking `Type` with it's nulls

In [33]:
df["Type"].unique()

array(['Free', 'Paid', nan], dtype=object)

In [34]:
df.loc[df["Type"].isna()]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,min android ver,installs range
8667,Command & Conquer: Rivals,FAMILY,NaN,0,NaN,0,NaN,0.0,Everyone 10+,Strategy,2018-06-28,Varies with device,NaN,NaN,0


##### Checking `Content Rating` with it's nulls

In [35]:
df["Content Rating"].unique()

array(['Everyone', 'Teen', 'Everyone 10+', 'Mature 17+',
       'Adults only 18+', 'Unrated'], dtype=object)

##### Checking `Genres` with it's nulls

In [36]:
df["Genres"].unique()

array(['Art & Design', 'Art & Design;Pretend Play',
       'Art & Design;Creativity', 'Art & Design;Action & Adventure',
       'Auto & Vehicles', 'Beauty', 'Books & Reference', 'Business',
       'Comics', 'Comics;Creativity', 'Communication', 'Dating',
       'Education;Education', 'Education', 'Education;Creativity',
       'Education;Music & Video', 'Education;Action & Adventure',
       'Education;Pretend Play', 'Education;Brain Games', 'Entertainment',
       'Entertainment;Music & Video', 'Entertainment;Brain Games',
       'Entertainment;Creativity', 'Events', 'Finance', 'Food & Drink',
       'Health & Fitness', 'House & Home', 'Libraries & Demo',
       'Lifestyle', 'Lifestyle;Pretend Play',
       'Adventure;Action & Adventure', 'Arcade', 'Casual', 'Card',
       'Casual;Pretend Play', 'Action', 'Strategy', 'Puzzle', 'Sports',
       'Music', 'Word', 'Racing', 'Casual;Creativity',
       'Casual;Action & Adventure', 'Simulation', 'Adventure', 'Board',
       'Trivia', 'Role 

In [37]:
df.loc[df["Genres"].isna()]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,min android ver,installs range
9990,Life Made WI-Fi Touchscreen Photo Frame,NaN,1.9,19,3.0,"1,000+",Free,0.0,Everyone,NaN,2018-02-11,1.0.19,4.0 and up,4.0,1000


> Converting `Genres` column to hold lists of genres since they are separated with ';'

In [38]:
df["Genres"] = df["Genres"].str.split(';')
df["Genres"].sample(20)

884                       [Events]
6055            [Libraries & Demo]
6032                      [Sports]
9458           [Maps & Navigation]
5575                     [Finance]
142            [Books & Reference]
3270     [Video Players & Editors]
3469                      [Sports]
7298                  [Simulation]
7893                    [Business]
2479                 [Photography]
7390                     [Medical]
928                      [Finance]
5670             [Auto & Vehicles]
1690                      [Action]
2329                      [Social]
1524                      [Arcade]
9471           [Maps & Navigation]
10232                     [Social]
4650                    [Business]
Name: Genres, dtype: object

> Dropping the nulls of these columns since they have only one null per column

In [39]:
df = df.dropna(subset=(['Category', "Type", "Genres"]))

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10356 entries, 0 to 10357
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   App              10356 non-null  object        
 1   Category         10356 non-null  object        
 2   Rating           8892 non-null   float64       
 3   Reviews          10356 non-null  int64         
 4   Size             8831 non-null   float64       
 5   Installs         10356 non-null  object        
 6   Type             10356 non-null  object        
 7   Price            10356 non-null  float64       
 8   Content Rating   10356 non-null  object        
 9   Genres           10356 non-null  object        
 10  Last Updated     10356 non-null  datetime64[ns]
 11  Current Ver      10348 non-null  object        
 12  Android Ver      9134 non-null   object        
 13  min android ver  9134 non-null   float64       
 14  installs range   10356 non-null  int64     

In [41]:
df["Current Ver"].sample(20)

1627                 2.21.1
6766               1.0.9.10
5033                   3.30
3697                      1
1850                    149
3345                  3.1.9
4492                  1.0.5
10263                 1.0.2
2772                 4.0.21
3653                   1.09
6504               6.6.3106
5                       1.0
7888                    2.0
10304                   1.2
9077                    2.4
1188     Varies with device
4891                    1.1
4664                  1.0.2
3362                  9.2.3
4145     Varies with device
Name: Current Ver, dtype: object

In [42]:
df["Current Ver"] = df["Current Ver"].replace("Varies with device", np.nan)